In [ ]:
import pandas as pd
import numpy as np

# Map Cluster outside Study Area to inside major transportation connectors

In [ ]:
csv_api = "traveltime_results_top1_all.csv"
df_api = pd.read_csv(csv_api)
df_api

In [ ]:
csv = "combined_data_MP_NE.csv"
df = pd.read_csv(csv)
df

In [ ]:
df['StartName'].unique()

In [ ]:
# Create a mapping dictionary from df_api
mapping_dict = df_api.set_index('search_id')['id'].to_dict()

# Apply the mapping to the StartName column in df, keeping original value if no match
df['StartName'] = df['StartName'].apply(lambda x: mapping_dict.get(x, x))
df

In [ ]:
df.StartName.unique()

In [ ]:
# Apply the mapping to the EndName column in df, keeping original value if no match
df['EndName'] = df['EndName'].apply(lambda x: mapping_dict.get(x, x))
df

In [ ]:
df.EndName.unique()

In [ ]:
df.StartId.unique()

In [ ]:
df.drop(columns=['StartId', 'EndId'], inplace=True)

In [ ]:
df_result = df.groupby(['Bucket', 'StartName', 'EndName', 'Wochentag', 'Tageszeit', 'Schulferien']).agg({
    'Count': 'sum',
    'Distanz': 'mean'
}).reset_index()
#df_result.rename(columns={'Distanz': 'mean_distance'}, inplace=True)
df_result

In [ ]:
df_result.Distanz.describe()

In [ ]:
duplicated_count = df_result.duplicated().sum()
duplicated_count

In [ ]:
df.Count.describe()

In [ ]:
df_result.Count.describe()

In [ ]:
# Count the occurrences where startClusterID and endClusterID are the same
same_cluster_count = df_result[df_result['StartName'] == df_result['EndName']].shape[0]
same_cluster_count

In [ ]:
# drop the rows where StartName is equal to EndName
df_result = df_result[df_result['StartName'] != df_result['EndName']]
df_result

In [ ]:
df_result.to_csv(csv.split('.')[0] + '_mappedUCOtoMTC.csv', index=False)

# Merge bottom 30% inside Study Area

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
xlsx = "mappingInsideStudyArea.xlsx"
df_xlsx = pd.read_excel(xlsx)
df_xlsx

In [ ]:
# Create a mapping dictionary from df_xlsx
mapping_dict_inside = df_xlsx.set_index('search_id')['id'].to_dict()
mapping_dict_inside

In [ ]:
csv_sa = "combined_data_MP_NE_dT_cC_Coord_hildesheimMerged.csv"
df_sa = pd.read_csv(csv_sa)
df_sa

In [ ]:
df_sa.distance.describe()

In [ ]:
df_sa.drop(columns=['startLon', 'startLat', 'endLon', 'endLat', 'startClusterName', 'startClusterZip', 'endClusterName', 'endClusterZip'], inplace=True)
df_sa

In [ ]:
df_sa.startClusterID.unique()

In [ ]:
# Apply the mapping to the column in df, keeping original value if no match
df_sa['startClusterID'] = df_sa['startClusterID'].apply(lambda x: mapping_dict_inside.get(x, x))
df_sa

In [ ]:
df_sa.startClusterID.unique()

In [ ]:
# Apply the mapping to the column in df, keeping original value if no match
df_sa['endClusterID'] = df_sa['endClusterID'].apply(lambda x: mapping_dict_inside.get(x, x))
df_sa

In [ ]:
df_sa.endClusterID.unique()

In [ ]:
df_sa_result = df_sa.groupby(['year', 'month', 'startClusterID', 'endClusterID', 'weekday', 'daytime', 'isSchoolHoliday']).agg({
    'count': 'sum',
    'count_corrected': 'sum',
    'distance': 'mean',
    'startClusterName': 'first',
    'startClusterZip': 'first',
    'startID': 'mean',
    'endClusterName': 'first',
    'endClusterZip': 'first',
    'endID': 'mean',
    'startLon': 'mean',
    'startLat': 'mean',
    'endLat': 'mean',
    'endLon': 'mean'
}).reset_index()
df_sa_result

In [ ]:
df_sa_result.rename(columns={'distance': 'mean_distance'}, inplace=True)
df_sa_result

In [ ]:
df_sa_result.mean_distance.describe()

In [ ]:
len(df_sa) - len(df_sa_result)

In [ ]:
duplicated_count = df_sa_result.duplicated().sum()
duplicated_count

In [ ]:
# Count the occurrences where startClusterID and endClusterID are the same
same_cluster_count = df_sa_result[df_sa_result['startClusterID'] == df_sa_result['endClusterID']].shape[0]
same_cluster_count

In [ ]:
df_sa_result.to_csv('combined_data_MP_NE_hildesheim_merged_dataTransformed_featureEngineered_mergedClusterInside.csv', index=False)

# merge endcluster into regions

In [ ]:
csv_shapes = "desd-4-landkreis-hildesheim-1663168323029-shapes.csv"
df_shapes = pd.read_csv(csv_shapes)
df_shapes

In [ ]:
df_shapes = df_shapes[df_shapes['name'].apply(lambda x: len(x) > 28)]
df_shapes['id'] = df_shapes['name'].apply(lambda x: x.split('- 0')[1][:-1])
df_shapes['id'] = df_shapes['id'].astype(int)
df_shapes

In [ ]:
csv = "combined_data_MP_NE_hildesheim_merged_dataTransformed_featureEngineered_mergedClusterInside_Coord_cC_start.csv"
df = pd.read_csv(csv)
df

In [ ]:
df_start = pd.DataFrame(df.startClusterID.unique())
df_start.rename(columns={0 : 'id'}, inplace=True)
df_start

In [ ]:
df_shapes = df_shapes.merge(df_start, on='id', how='right')
df_shapes

In [ ]:
df_shapes['name'] = df_shapes['name'].astype(str).apply(lambda x: x.split(' (')[0])
df_shapes

In [ ]:
df_shapes.drop(columns=['lon', 'lat', 'FID'], inplace=True)
df_shapes

In [ ]:
"""df_shapes.loc[df_shapes['id'] == 3241013050007, 'name'] = 'Pattensen/ZOB'
df_shapes.loc[df_shapes['id'] == 3254028001003, 'name'] = 'Sarstedt Train'
df_shapes.loc[df_shapes['id'] == 3254028001013, 'name'] = 'Sarstedt Tram'
df_shapes.loc[df_shapes['id'] == 3254014000004, 'name'] = 'Elze Train'
df_shapes.loc[df_shapes['id'] == 3254021003014, 'name'] = 'Hildesheim HBF'
df_shapes"""

In [ ]:
# Create a mapping dictionary from df_api
mapping_dict_regions = df_shapes.set_index('id')['name'].to_dict()
mapping_dict_regions

In [ ]:
csv = "combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd.csv"
df = pd.read_csv(csv)
df

In [ ]:
# Apply the mapping to the column in df, keeping original value if no match
df['endClusterID'] = df['endClusterID'].apply(lambda x: mapping_dict_regions.get(x, x))
df

In [ ]:
#df.drop(columns=['endLat', 'endLon'], inplace=True)

In [ ]:
df_result = df.groupby(['year', 'quarter','month', 'startClusterID', 'endClusterID', 'isSchoolHoliday','isWeekend', 'weekday_number', 'daytime', 'startLon', 'startLat']).agg({
    'endLon': 'mean',
    'endLat': 'mean',
    'mean_distance': 'mean',
    'count_corrected': 'sum',
}).reset_index()
df_result

In [ ]:
df_result.groupby(['endClusterID']).count()

In [ ]:
df_result.loc[df_result['endClusterID'] == 'Harsum', 'endClusterID'] = 'Giesen'
df_result.groupby(['endClusterID']).count()

In [ ]:
df_result

In [ ]:
df_result['count_corrected'].describe()

In [ ]:
df_result.to_csv(csv.split('.')[0] + '_regions_6.csv', index=False)